In [1]:
import requests
import polars as pl
from datetime import datetime, timedelta
import useful
import config
import pandas as pd
API_KEY = config.polygon_api1

In [2]:
#Get dataframe of all tickers
constituents = pl.read_csv("../Data/sp500/Constituents.csv", infer_schema_length=False)

#Create list of individual stocks from names from dataframe
stockComponents=[]
for k in range(len(constituents)):
    stockComponents.append(constituents[k,0])

In [3]:
def getStockData(stockComponent):
    ticker = stockComponent
    timespan = "minute"
    start_date = datetime(2019, 10, 1)
    end_date = datetime(2024, 9, 19)
    batch_size = 60
    current_date = start_date
    all_data = []

    while current_date < end_date:
        batch_end_date = current_date + timedelta(days=batch_size)
        if batch_end_date > end_date:
            batch_end_date = end_date
        url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/{timespan}/{current_date.date()}/{batch_end_date.date()}?adjusted=true&sort=asc&limit=50000&apiKey={API_KEY}"
        response = requests.get(url)
        data = response.json()
        if "results" in data:
            all_data.extend(data["results"])
        else:
            print(data)
        current_date = batch_end_date + timedelta(days=1)
    df = pd.DataFrame(all_data)
    df = df.rename(columns={"v":"volume", "vw":"weighted_volume", "o":"open", "c":"close", "h":"high", "l":"low", "t":"time", "n":"number_of_trades"})

    df['datetime_gmt'] = pd.to_datetime(df['time'], unit='ms', utc=True)
    df['datetime_est'] = df['datetime_gmt'].dt.tz_convert('US/Eastern')
    df['date'] = df['datetime_est'].dt.date
    df['time_gmt'] = df['datetime_gmt'].dt.time
    df['time_est'] = df['datetime_est'].dt.tz_convert('US/Eastern').dt.time
    df['day_of_week'] = df['datetime_est'].dt.day_name()
    df['week'] = df['datetime_est'].dt.isocalendar().week
    df['month'] = df['datetime_est'].dt.month
    df['trading_minute'] = df['datetime_est'].dt.hour * 60 + df['datetime_est'].dt.minute - 570
    df['trading_hours'] = ((df['datetime_est'].dt.hour == 9) & (df['datetime_est'].dt.minute > 30)) | ((df['datetime_est'].dt.hour > 9) & (df['datetime_est'].dt.hour < 16))
    df=df.drop(['datetime_est', 'datetime_gmt'], axis=1)
    df.to_csv('../Data/SP500/minuteHist2019/'+ticker+'.csv', index=False)

In [28]:
for k in range(len(stockComponents)):
    ticker = stockComponents[k]
    timespan = "minute"
    start_date = datetime(2019, 10, 1)
    end_date = datetime(2024, 9, 19)
    batch_size = 60
    current_date = start_date
    all_data = []

    while current_date < end_date:
        batch_end_date = current_date + timedelta(days=batch_size)
        if batch_end_date > end_date:
            batch_end_date = end_date
        url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/{timespan}/{current_date.date()}/{batch_end_date.date()}?adjusted=true&sort=asc&limit=50000&apiKey={API_KEY}"
        response = requests.get(url)
        data = response.json()
        if "results" in data:
            all_data.extend(data["results"])
        else:
            print(data)
        current_date = batch_end_date + timedelta(days=1)
        
    df = pd.DataFrame(all_data)
    df = df.rename(columns={"v":"volume", "vw":"weighted_volume", "o":"open", "c":"close", "h":"high", "l":"low", "t":"time", "n":"number_of_trades"})

    df['datetime_gmt'] = pd.to_datetime(df['time'], unit='ms', utc=True)
    df['datetime_est'] = df['datetime_gmt'].dt.tz_convert('US/Eastern')
    df['date'] = df['datetime_est'].dt.date
    df['time_gmt'] = df['datetime_gmt'].dt.time
    df['time_est'] = df['datetime_est'].dt.tz_convert('US/Eastern').dt.time
    df['day_of_week'] = df['datetime_est'].dt.day_name()
    df['week'] = df['datetime_est'].dt.isocalendar().week
    df['month'] = df['datetime_est'].dt.month
    df['trading_minute'] = df['datetime_est'].dt.hour * 60 + df['datetime_est'].dt.minute - 570
    df['trading_hours'] = ((df['datetime_est'].dt.hour == 9) & (df['datetime_est'].dt.minute >= 30)) | ((df['datetime_est'].dt.hour > 9) & (df['datetime_est'].dt.hour < 16))
    df=df.drop(['datetime_est', 'datetime_gmt'], axis=1)

    df.to_csv('../Data/SP500/minuteHist2019/'+ticker+'.csv', index=False)

{'ticker': 'ABC', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': 'c0149d17da59d36ad95254e1478b0682'}
{'ticker': 'ABC', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': 'c131bb66b2cd666607546e14637bcc03'}
{'ticker': 'ABC', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '6ef0f74ccbdcc6f8a76caecff159d647'}
{'ticker': 'ABC', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '8e0f2ce5bd0415c1156b2f4a3b21e6e9'}
{'ticker': 'ABC', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': 'da4ffdd24dc03d28f43c32c2ced2c280'}
{'ticker': 'ABC', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '98a663300b53d6f58d2ef69d2f5e4d40'}
{'ticker': 'ATVI', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '746ce7a6855861ec2b6e6f7310b85810'}
{'ticker': 'ATVI', 'queryCount': 

In [4]:
current=pd.read_csv('../Data/SP500/minuteHist2019/NVDA.csv')
current

,volume,weighted_volume,open,close,high,low,time,number_of_trades,date,time_gmt,time_est,day_of_week,week,month,trading_minute,trading_hours,year
0,20000.0,4.3875,4.3875,4.3875,4.3875,4.3875,1569916980000,6,2019-10-01,08:03:00,04:03:00,Tuesday,40,10,-327,False,2019
1,20000.0,4.3875,4.3875,4.3875,4.3875,4.3875,1569917280000,2,2019-10-01,08:08:00,04:08:00,Tuesday,40,10,-322,False,2019
2,20000.0,4.3545,4.3540,4.3540,4.3540,4.3540,1569922740000,5,2019-10-01,09:39:00,05:39:00,Tuesday,40,10,-231,False,2019
3,17600.0,4.3750,4.3750,4.3750,4.3750,4.3750,1569924600000,1,2019-10-01,10:10:00,06:10:00,Tuesday,40,10,-200,False,2019
4,5240.0,4.3778,4.3788,4.3788,4.3788,4.3788,1569925020000,3,2019-10-01,10:17:00,06:17:00,Tuesday,40,10,-193,False,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883375,15491.0,117.8930,117.9000,117.9000,117.9200,117.8700,1726790100000,96,2024-09-19,23:55:00,19:55:00,Thursday,38,9,625,False,2024
883376,6989.0,117.8856,117.8700,117.8900,117.8990,117.8700,1726790160000,93,2024-09-19,23:56:00,19:56:00,Thursday,38,9,626,False,2024
883377,14848.0,117.8589,117.8700,117.8505,117.8800,117.8400,1726790220000,124,2024-09-19,23:57:00,19:57:00,Thursday,38,9,627,False,2024
883378,17821.0,117.8616,117.8500,117.8900,117.8900,117.8500,1726790280000,115,2024-09-19,23:58:00,19:58:00,Thursday,38,9,628,False,2024


In [6]:
for k in range(len(stockComponents)):
    ticker = stockComponents[k]
    df = pd.read_csv(f'../Data/SP500/minuteHist2019/{ticker}.csv', index_col=False)
    df['datetime_gmt'] = pd.to_datetime(df['time'], unit='ms', utc=True)
    df['datetime_est'] = df['datetime_gmt'].dt.tz_convert('US/Eastern')
    df['trading_hours'] = ((df['datetime_est'].dt.hour == 9) & (df['datetime_est'].dt.minute >= 30)) | ((df['datetime_est'].dt.hour > 9) & (df['datetime_est'].dt.hour < 16))
    df.to_csv(f'../Data/SP500/minuteHist2019/{ticker}.csv', index=False)

In [17]:
df = pd.read_csv(f'../Data/SP500/minuteHist2019/TSLA.csv')

In [18]:
df

,volume,weighted_volume,open,close,high,low,time,number_of_trades,date,time_gmt,time_est,day_of_week,week,month,trading_minute,trading_hours
0,4500.0,16.1233,16.1233,16.1233,16.1233,16.1233,1569920760000,3,2019-10-01,09:06:00,05:06:00,Tuesday,40,10,-264,False
1,8640.0,16.1295,16.1300,16.1293,16.1300,16.1293,1569921600000,3,2019-10-01,09:20:00,05:20:00,Tuesday,40,10,-250,False
2,2625.0,16.1293,16.1293,16.1293,16.1293,16.1293,1569923580000,1,2019-10-01,09:53:00,05:53:00,Tuesday,40,10,-217,False
3,15615.0,16.1337,16.1347,16.1333,16.1347,16.1333,1569927780000,3,2019-10-01,11:03:00,07:03:00,Tuesday,40,10,-147,False
4,3000.0,16.1340,16.1333,16.1333,16.1333,16.1333,1569927900000,4,2019-10-01,11:05:00,07:05:00,Tuesday,40,10,-145,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002194,9569.0,244.0298,244.0200,244.0000,244.1000,244.0000,1726790100000,97,2024-09-19,23:55:00,19:55:00,Thursday,38,9,625,False
1002195,17961.0,244.0294,244.0000,244.1000,244.1000,244.0000,1726790160000,126,2024-09-19,23:56:00,19:56:00,Thursday,38,9,626,False
1002196,8199.0,244.1266,244.1000,244.1435,244.1900,244.1000,1726790220000,125,2024-09-19,23:57:00,19:57:00,Thursday,38,9,627,False
1002197,14435.0,244.0643,244.1400,244.0000,244.1500,244.0000,1726790280000,133,2024-09-19,23:58:00,19:58:00,Thursday,38,9,628,False
